In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display

# Simulation with sinusoidal death-rate modulation for W and V or for Y
def simulate_and_plot(
    V0, W0, Y0, X0, Z0,
    W_birth, Y_birth, W_death, Y_death,
    X_in, Z_in, X_out, Z_out,
    Time, use_X, use_Z, show_phase_only,
    severity, period, affect_W):
    # Scaling factors for X and Z
    X_scaler = X_out / X_in
    Z_scaler = Z_out / Z_in

    # Equilibrium values for W and Y (base rates)
    Q1 = W_death / W_birth
    Q2 = Y_death / Y_birth
    disc_W = (1 - Q1 + Q2)**2 - 4 * Q2
    if disc_W >= 0:
        sqrt_disc = np.sqrt(disc_W)
        W_eq1 = 0.5 * ((1 - Q1 + Q2) + sqrt_disc)
        W_eq2 = 0.5 * ((1 - Q1 + Q2) - sqrt_disc)
    else:
        W_eq1 = W_eq2 = np.nan
    disc_Y = (1 - Q2 + Q1)**2 - 4 * Q1
    if disc_Y >= 0:
        sqrt_disc = np.sqrt(disc_Y)
        Y_eq1 = 0.5 * ((1 - Q2 + Q1) + sqrt_disc)
        Y_eq2 = 0.5 * ((1 - Q2 + Q1) - sqrt_disc)
    else:
        Y_eq1 = Y_eq2 = np.nan

    # Time vector
    dt = 0.1
    t = np.arange(0, Time + dt, dt)
    n_steps = len(t)

    # Angular frequency for sinusoid
    omega = 2 * np.pi / period

    # Initialize populations
    V = np.zeros(n_steps)
    W = np.zeros(n_steps)
    Y = np.zeros(n_steps)
    X = np.zeros(n_steps)
    Z = np.zeros(n_steps)
    V[0], W[0], Y[0], X[0], Z[0] = V0, W0, Y0, X0 / X_scaler, Z0 / Z_scaler

    # Choose iterator with optional progress bar
    iterator = range(1, n_steps)
    if n_steps > 200_000:
        iterator = tqdm(iterator, desc="Simulating")

    # Simulation loop
    for i in iterator:
        # Determine time-varying death rates
        if affect_W:
            D_WV = W_death * (1 + severity * np.sin(omega * t[i-1]))
            D_Y   = Y_death
        else:
            D_WV = W_death
            D_Y   = Y_death * (1 + severity * np.sin(omega * t[i-1]))

        # Compute derivatives
        dV =  W_birth * (1 - W[i-1] - V[i-1]) * V[i-1] * Y[i-1] - D_WV * V[i-1]
        dW =  W_birth * (1 - W[i-1] - V[i-1]) * W[i-1] * Y[i-1] - D_WV * W[i-1]
        dY =  Y_birth * (1 - Y[i-1]) * Y[i-1] * (V[i-1] + W[i-1]) - D_Y * Y[i-1]

        # Exchange terms
        if use_X:
            dW += X_out * X[i-1] - X_in * W[i-1]
        if use_Z:
            dY += Z_out * Z[i-1] - Z_in * Y[i-1]
        dX = -X_out * X[i-1] + X_in * W[i-1]
        dZ = -Z_out * Z[i-1] + Z_in * Y[i-1]

        # Euler update
        V[i] = V[i-1] + dt * dV
        W[i] = W[i-1] + dt * dW
        Y[i] = Y[i-1] + dt * dY
        X[i] = X[i-1] + dt * dX
        Z[i] = Z[i-1] + dt * dZ

    # Prepare for plotting
    X_plot = X * X_scaler
    Z_plot = Z * Z_scaler

    # Plot time series
    if not show_phase_only:
        plt.figure(figsize=(12, 5))
        plt.plot(t, Y, label=r'$Y_t$', color='darkblue')
        if use_X: plt.plot(t, X_plot, label=r'$X_t$', color='lightgreen')
        if use_Z: plt.plot(t, Z_plot, label=r'$Z_t$', color='skyblue')
        plt.plot(t, V, label=r'$V_t$', color='orange')
        plt.plot(t, W, label=r'$W_t$', color='darkgreen')
        # Equilibrium lines for W and Y
        if not np.isnan(W_eq1): plt.axhline(W_eq1, linestyle='--', label=r'$W^+_{eq}$')
        if not np.isnan(W_eq2): plt.axhline(W_eq2, linestyle='--', label=r'$W^-_{eq}$')
        if not np.isnan(Y_eq1): plt.axhline(Y_eq1, linestyle=':', label=r'$Y^+_{eq}$')
        if not np.isnan(Y_eq2): plt.axhline(Y_eq2, linestyle=':', label=r'$Y^-_{eq}$')
        plt.xlabel('Time')
        plt.ylabel('Population')
        plt.title('Population Dynamics Over Time')
        plt.ylim(0, 1)
        plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
        plt.tight_layout()
        plt.savefig('population_dynamics.pdf')
        plt.show()

    # Plot phase space
    else:
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))
        axs[0].plot(W, Y, label='W vs Y', color='purple')
        if use_X and use_Z:
            axs[0].plot(X, Z, label='X vs Z', color='brown', linestyle='--')
        if not np.isnan(W_eq1) and not np.isnan(Y_eq1):
            axs[0].scatter(W_eq1, Y_eq1, marker='o', label='Equilibrium +')
        if not np.isnan(W_eq2) and not np.isnan(Y_eq2):
            axs[0].scatter(W_eq2, Y_eq2, marker='x', label='Equilibrium -')
        axs[0].set(xlabel='W', ylabel='Y', xlim=(0,1), ylim=(0,1), title='Phase Plot')
        axs[0].grid(True)
        axs[0].legend(loc='best')
        Q_vals = np.linspace(0,1,400)
        q1g, q2g = np.meshgrid(Q_vals, Q_vals)
        valid = (np.sqrt(q1g) + np.sqrt(q2g)) <= 1
        axs[1].contourf(Q_vals, Q_vals, valid, levels=[0.5,1], alpha=0.5)
        axs[1].plot(Q_vals, (1 - np.sqrt(Q_vals))**2, 'k--')
        axs[1].scatter(Q1, Q2, color='red', zorder=5)
        axs[1].text(Q1+0.02, Q2+0.02, f"$\sqrt{{Q1}}+\sqrt{{Q2}}={(np.sqrt(Q1)+np.sqrt(Q2)):.2f}$")
        axs[1].set(xlabel='Q1', ylabel='Q2', xlim=(0,1), ylim=(0,1), title='Constraint Region')
        plt.tight_layout()
        plt.savefig('phase_population_dynamics.pdf')
        plt.show()

    # Return arrays for post-processing
    return t, V, W, Y, X, Z

# Widget definitions
slider_layout = widgets.Layout(width='240px', margin='2px 2px 2px -20px')

V0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.47, description='V0', layout=slider_layout)
W0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.2,  description='W0', layout=slider_layout)
Y0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.75, description='Y0', layout=slider_layout)
X0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.2,  description='X0', layout=slider_layout)
Z0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.3,  description='Z0', layout=slider_layout)

W_birth_slider = widgets.FloatSlider(min=0.01, max=2.0, step=0.01, value=0.4,  description='W_birth', layout=slider_layout)
Y_birth_slider = widgets.FloatSlider(min=0.01, max=2.0, step=0.01, value=0.9,  description='Y_birth', layout=slider_layout)
W_death_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1,  description='W_death', layout=slider_layout)
Y_death_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.15, description='Y_death', layout=slider_layout)

X_in_slider  = widgets.FloatSlider(min=0.01, max=2.0, step=0.01, value=0.2,  description='X_in', layout=slider_layout)
X_out_slider = widgets.FloatSlider(min=0.0,  max=2.0, step=0.01, value=0.1,  description='X_out', layout=slider_layout)
Z_in_slider  = widgets.FloatSlider(min=0.01, max=2.0, step=0.01, value=0.2,  description='Z_in', layout=slider_layout)
Z_out_slider = widgets.FloatSlider(min=0.0,  max=2.0, step=0.01, value=0.05, description='Z_out', layout=slider_layout)

Time_slider = widgets.IntSlider(min=10, max=50000, step=10, value=400, description='Time', layout=slider_layout)
phase_only_checkbox = widgets.Checkbox(value=False, description='Phase Only', layout=slider_layout)
use_X = widgets.Checkbox(value=True, description='Enable X', layout=slider_layout)
use_Z = widgets.Checkbox(value=False, description='Enable Z', layout=slider_layout)

severity_slider = widgets.FloatSlider(min=0, max=0.99, step=0.01, value=0.1, description='Severity', layout=slider_layout)
period_slider = widgets.IntSlider(min=1, max=50, step=1, value=30, description='Period', layout=slider_layout)
affect_W_checkbox = widgets.Checkbox(value=False, description='Affect W death', layout=slider_layout)

# Layout controls
col1 = widgets.VBox([V0_slider, W0_slider, Y0_slider, X0_slider, Z0_slider])
col2 = widgets.VBox([W_birth_slider, W_death_slider, Y_birth_slider, Y_death_slider])
col3 = widgets.VBox([X_in_slider, X_out_slider, Z_in_slider, Z_out_slider])
col4 = widgets.VBox([Time_slider, phase_only_checkbox, use_X, use_Z])
col5 = widgets.VBox([severity_slider, period_slider, affect_W_checkbox])
controls = widgets.HBox([col1, col2, col3, col4, col5])

# Bind and display
out = widgets.interactive_output(
    simulate_and_plot,
    {
        'V0': V0_slider, 'W0': W0_slider, 'Y0': Y0_slider, 'X0': X0_slider, 'Z0': Z0_slider,
        'W_birth': W_birth_slider, 'Y_birth': Y_birth_slider,
        'W_death': W_death_slider, 'Y_death': Y_death_slider,
        'X_in': X_in_slider, 'Z_in': Z_in_slider,
        'X_out': X_out_slider, 'Z_out': Z_out_slider,
        'Time': Time_slider,
        'use_X': use_X, 'use_Z': use_Z, 'show_phase_only': phase_only_checkbox,
        'severity': severity_slider, 'period': period_slider, 'affect_W': affect_W_checkbox
    }
)

display(controls, out)

Output()

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display

# Simulation with inverted half-sinusoidal death-rate modulation for W/V or for Y
# f(t) = -cos(pi * (t % period) / period): goes from -1 to +1 smoothly, then jumps from +1 to -1
# at the start of each period.
def simulate_and_plot(
    V0, W0, Y0, X0, Z0,
    W_birth, Y_birth, W_death, Y_death,
    X_in, Z_in, X_out, Z_out,
    Time, use_X, use_Z, show_phase_only,
    severity, period, affect_W
):
    # Scaling factors for X and Z
    X_scaler = X_out / X_in
    Z_scaler = Z_out / Z_in

    # Equilibrium values for W and Y (base rates)
    Q1 = W_death / W_birth
    Q2 = Y_death / Y_birth
    disc_W = (1 - Q1 + Q2)**2 - 4 * Q2
    if disc_W >= 0:
        sqrt_disc = np.sqrt(disc_W)
        W_eq1 = 0.5 * ((1 - Q1 + Q2) + sqrt_disc)
        W_eq2 = 0.5 * ((1 - Q1 + Q2) - sqrt_disc)
    else:
        W_eq1 = W_eq2 = np.nan
    disc_Y = (1 - Q2 + Q1)**2 - 4 * Q1
    if disc_Y >= 0:
        sqrt_disc = np.sqrt(disc_Y)
        Y_eq1 = 0.5 * ((1 - Q2 + Q1) + sqrt_disc)
        Y_eq2 = 0.5 * ((1 - Q2 + Q1) - sqrt_disc)
    else:
        Y_eq1 = Y_eq2 = np.nan

    # Time vector and step size
    dt = 0.01
    t = np.arange(0, Time + dt, dt)
    n_steps = len(t)

    # Initialize populations
    V = np.zeros(n_steps)
    W = np.zeros(n_steps)
    Y = np.zeros(n_steps)
    X = np.zeros(n_steps)
    Z = np.zeros(n_steps)
    V[0], W[0], Y[0], X[0], Z[0] = V0, W0, Y0, X0 / X_scaler, Z0 / Z_scaler

    # Iterator with optional tqdm
    iterator = range(1, n_steps)
    if n_steps > 200_000:
        iterator = tqdm(iterator, desc="Simulating")

    # Simulation loop
    for i in iterator:
        # Compute inverted half-sinus waveform via cosine
        tau = t[i-1] % period
        f = -np.cos(np.pi * tau / period)

        # Determine time-varying death rates
        if affect_W:
            D_WV = W_death * (1 + severity * f)
            D_Y   = Y_death
        else:
            D_WV = W_death
            D_Y   = Y_death * (1 + severity * f)

        # Compute derivatives
        dV = W_birth * (1 - W[i-1] - V[i-1]) * V[i-1] * Y[i-1] - D_WV * V[i-1]
        dW = W_birth * (1 - W[i-1] - V[i-1]) * W[i-1] * Y[i-1] - D_WV * W[i-1]
        dY = Y_birth * (1 - Y[i-1]) * Y[i-1] * (V[i-1] + W[i-1]) - D_Y * Y[i-1]

        # Exchange terms
        if use_X:
            dW += X_out * X[i-1] - X_in * W[i-1]
        if use_Z:
            dY += Z_out * Z[i-1] - Z_in * Y[i-1]
        dX = -X_out * X[i-1] + X_in * W[i-1]
        dZ = -Z_out * Z[i-1] + Z_in * Y[i-1]

        # Euler update
        V[i] = V[i-1] + dt * dV
        W[i] = W[i-1] + dt * dW
        Y[i] = Y[i-1] + dt * dY
        X[i] = X[i-1] + dt * dX
        Z[i] = Z[i-1] + dt * dZ

    # Prepare for plotting
    X_plot = X * X_scaler
    Z_plot = Z * Z_scaler

    # Plot time series
    if not show_phase_only:
        plt.figure(figsize=(12, 5))
        plt.plot(t, Y, label=r'$Y_t$', color='darkblue')
        if use_X: plt.plot(t, X_plot, label=r'$X_t$', color='lightgreen')
        if use_Z: plt.plot(t, Z_plot, label=r'$Z_t$', color='skyblue')
        plt.plot(t, V, label=r'$V_t$', color='orange')
        plt.plot(t, W, label=r'$W_t$', color='darkgreen')
        # Equilibrium lines for W and Y
        if not np.isnan(W_eq1): plt.axhline(W_eq1, linestyle='--', label=r'$W^+_{eq}$')
        if not np.isnan(W_eq2): plt.axhline(W_eq2, linestyle='--', label=r'$W^-_{eq}$')
        if not np.isnan(Y_eq1): plt.axhline(Y_eq1, linestyle=':', label=r'$Y^+_{eq}$')
        if not np.isnan(Y_eq2): plt.axhline(Y_eq2, linestyle=':', label=r'$Y^-_{eq}$')
        plt.xlabel('Time')
        plt.ylabel('Population')
        plt.title('Population Dynamics Over Time')
        plt.ylim(0, 1)
        plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
        plt.tight_layout()
        plt.savefig('population_dynamics.pdf')
        plt.show()

    # Plot phase space
    else:
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))
        axs[0].plot(W, Y, label='W vs Y', color='purple')
        if use_X and use_Z:
            axs[0].plot(X, Z, label='X vs Z', color='brown', linestyle='--')
        if not np.isnan(W_eq1) and not np.isnan(Y_eq1):
            axs[0].scatter(W_eq1, Y_eq1, marker='o', label='Equilibrium +')
        if not np.isnan(W_eq2) and not np.isnan(Y_eq2):
            axs[0].scatter(W_eq2, Y_eq2, marker='x', label='Equilibrium -')
        axs[0].set(xlabel='W', ylabel='Y', xlim=(0,1), ylim=(0,1), title='Phase Plot')
        axs[0].grid(True)
        axs[0].legend(loc='best')
        Q_vals = np.linspace(0,1,400)
        q1g, q2g = np.meshgrid(Q_vals, Q_vals)
        valid = (np.sqrt(q1g) + np.sqrt(q2g)) <= 1
        axs[1].contourf(Q_vals, Q_vals, valid, levels=[0.5,1], alpha=0.5)
        axs[1].plot(Q_vals, (1 - np.sqrt(Q_vals))**2, 'k--')
        axs[1].scatter(Q1, Q2, color='red', zorder=5)
        axs[1].text(Q1+0.02, Q2+0.02, f"$\sqrt{{Q1}}+\sqrt{{Q2}}={(np.sqrt(Q1)+np.sqrt(Q2)):.2f}$")
        axs[1].set(xlabel='Q1', ylabel='Q2', xlim=(0,1), ylim=(0,1), title='Constraint Region')
        plt.tight_layout()
        plt.savefig('phase_population_dynamics.pdf')
        plt.show()

    # Return arrays for post-processing
    return t, V, W, Y, X, Z

# Widget definitions
slider_layout = widgets.Layout(width='240px', margin='2px 2px 2px -20px')

V0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.47, description='V0', layout=slider_layout)
W0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.2,  description='W0', layout=slider_layout)
Y0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.75, description='Y0', layout=slider_layout)
X0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.2,  description='X0', layout=slider_layout)
Z0_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.3,  description='Z0', layout=slider_layout)

W_birth_slider = widgets.FloatSlider(min=0.01, max=2.0, step=0.01, value=0.4,  description='W_birth', layout=slider_layout)
Y_birth_slider = widgets.FloatSlider(min=0.01, max=2.0, step=0.01, value=0.9,  description='Y_birth', layout=slider_layout)
W_death_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.1,  description='W_death', layout=slider_layout)
Y_death_slider = widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.15, description='Y_death', layout=slider_layout)

X_in_slider  = widgets.FloatSlider(min=0.01, max=2.0, step=0.01, value=0.2,  description='X_in', layout=slider_layout)
X_out_slider = widgets.FloatSlider(min=0.0,  max=2.0, step=0.01, value=0.1,  description='X_out', layout=slider_layout)
Z_in_slider  = widgets.FloatSlider(min=0.01, max=2.0, step=0.01, value=0.2,  description='Z_in', layout=slider_layout)
Z_out_slider = widgets.FloatSlider(min=0.0,  max=2.0, step=0.01, value=0.05, description='Z_out', layout=slider_layout)

Time_slider = widgets.IntSlider(min=10, max=50000, step=10, value=400, description='Time', layout=slider_layout)
phase_only_checkbox = widgets.Checkbox(value=False, description='Phase Only', layout=slider_layout)
use_X = widgets.Checkbox(value=True, description='Enable X', layout=slider_layout)
use_Z = widgets.Checkbox(value=False, description='Enable Z', layout=slider_layout)

severity_slider = widgets.FloatSlider(min=0, max=0.99, step=0.01, value=0.1, description='Severity', layout=slider_layout)
period_slider = widgets.IntSlider(min=1, max=50, step=1, value=30, description='Period', layout=slider_layout)
affect_W_checkbox = widgets.Checkbox(value=False, description='Affect W death', layout=slider_layout)

# Layout controls
col1 = widgets.VBox([V0_slider, W0_slider, Y0_slider, X0_slider, Z0_slider])
col2 = widgets.VBox([W_birth_slider, W_death_slider, Y_birth_slider, Y_death_slider])
col3 = widgets.VBox([X_in_slider, X_out_slider, Z_in_slider, Z_out_slider])
col4 = widgets.VBox([Time_slider, phase_only_checkbox, use_X, use_Z])
col5 = widgets.VBox([severity_slider, period_slider, affect_W_checkbox])
controls = widgets.HBox([col1, col2, col3, col4, col5])

# Bind and display
out = widgets.interactive_output(
    simulate_and_plot,
    {
        'V0': V0_slider, 'W0': W0_slider, 'Y0': Y0_slider, 'X0': X0_slider, 'Z0': Z0_slider,
        'W_birth': W_birth_slider, 'Y_birth': Y_birth_slider,
        'W_death': W_death_slider, 'Y_death': Y_death_slider,
        'X_in': X_in_slider, 'Z_in': Z_in_slider,
        'X_out': X_out_slider, 'Z_out': Z_out_slider,
        'Time': Time_slider,
        'use_X': use_X, 'use_Z': use_Z, 'show_phase_only': phase_only_checkbox,
        'severity': severity_slider, 'period': period_slider, 'affect_W': affect_W_checkbox
    }
)

display(controls, out)




Output()